In [14]:
from numpy import char
from datetime import date
import pandas as pd
import numpy as np
import time
pd.set_option('display.float_format', str)

In [15]:
l_columnnames = ["L_ORDERKEY", "L_PARTKEY", "L_SUPPKEY", "L_LINENUMBER", "L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_TAX", "L_RETURNFLAG", "L_LINESTATUS", "L_SHIPDATE", "L_COMMITDATE", "L_RECEIPTDATE", "L_SHIPINSTRUCT","L_SHIPMODE", "L_COMMENT"]

for i in range(len(l_columnnames)):
    l_columnnames[i] = l_columnnames[i].lower()
    
l_data_types = {
    'l_orderkey': int,
    'l_partkey': int,
    'l_suppkey': int,
    'l_linenumber': int,
    'l_quantity': float,
    'l_extendedprice': float,
    'l_discount': float,
    'l_tax': float,
    'l_returnflag': str,
    'l_linestatus': str,
    'l_shipinstruct': str,
    'l_shipmode': str,
    'l_comment': str
}

l_parse_dates = ['l_shipdate', 'l_commitdate', 'l_receiptdate']

In [16]:
o_columnnames = ["O_ORDERKEY", "O_CUSTKEY", "O_ORDERSTATUS", "O_TOTALPRICE", "O_ORDERDATE", "O_ORDERPRIORITY", "O_CLERK", "O_SHIPPRIORITY", "O_COMMENT"]

for i in range(len(o_columnnames)):
    o_columnnames[i] = o_columnnames[i].lower()
    
o_data_types = {
    'o_orderkey': int,
    'o_custkey': int,
    'o_orderstatus': str,
    'o_totalprice': float,
    'o_orderpriority': str,
    'o_clerk': str,
    'o_shippriority': int,
    'o_comment': str
}

o_parse_dates = ['o_orderdate']

In [17]:
c_columnnames = ["C_CUSTKEY", "C_NAME", "C_ADDRESS", "C_NATIONKEY", "C_PHONE", "C_ACCTBAL", "C_MKTSEGMENT", "C_COMMENT"]

for i in range(len(c_columnnames)):
    c_columnnames[i] = c_columnnames[i].lower()
    
c_data_types = {
    'c_custkey': int,
    'c_name': str,
    'c_address': str,
    'c_nationkey': int,
    'c_phone': str,
    'c_acctbal': float,
    'c_mktsegment': str,
    'c_comment': str
}

c_parse_dates = []

In [18]:
# Don't set indexes, as we can't access them with Pandas selection!
lineitem = pd.read_table("../tpch-pgsql-master/data/load/lineitem.tbl.csv", sep="|", names=l_columnnames, dtype=l_data_types, parse_dates=l_parse_dates)

In [19]:
# Don't set indexes, as we can't access them with Pandas selection!
orders = pd.read_table("../tpch-pgsql-master/data/load/orders.tbl.csv", sep="|", names=o_columnnames, dtype=o_data_types, parse_dates=o_parse_dates)

In [20]:
# Don't set indexes, as we can't access them with Pandas selection!
customer = pd.read_table("../tpch-pgsql-master/data/load/customer.tbl.csv", sep="|", names=c_columnnames, dtype=c_data_types, parse_dates=c_parse_dates)

In [21]:
start_t = time.time()

df_filter_1 = customer[['c_custkey', 'c_name', 'c_address', 'c_nationkey', 'c_phone', 'c_acctbal', 'c_mktsegment', 'c_comment']]
df_filter_2 = orders[['o_orderkey', 'o_orderdate', 'o_totalprice', 'o_custkey']]
df_filter_3 = lineitem[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
df_group_1 = df_filter_3 \
    .groupby(['l_orderkey']) \
    .agg(
    )
df_group_1 = df_group_1[['l_orderkey']]
df_merge_1 = df_filter_2.merge(df_group_1, left_on="o_orderkey", right_on="l_orderkey")
df_merge_1 = df_merge_1[['o_orderkey', 'o_orderdate', 'o_totalprice', 'o_custkey', 'lineitem_1.l_orderkey']]
df_sort_1 = df_merge_1.sort_values(by=['o_custkey'], ascending=[True])
df_sort_1 = df_sort_1[['o_orderkey', 'o_orderdate', 'o_totalprice', 'o_custkey', 'lineitem_1.l_orderkey']]
df_merge_2 = df_filter_1.merge(df_sort_1, left_on="c_custkey", right_on="o_custkey")
df_merge_2 = df_merge_2[['c_name', 'c_custkey', 'o_orderkey', 'o_orderdate', 'o_totalprice', 'lineitem_1.l_orderkey']]
df_filter_4 = lineitem[['l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
df_merge_3 = df_merge_2.merge(df_filter_4, left_on="o_orderkey", right_on="l_orderkey")
df_merge_3 = df_merge_3[['c_custkey', 'o_orderkey', 'c_name', 'o_orderdate', 'o_totalprice', 'l_quantity']]
df_sort_2 = df_merge_3.sort_values(by=['c_custkey', 'o_orderkey'], ascending=[True, True])
df_sort_2 = df_sort_2[['c_custkey', 'o_orderkey', 'c_name', 'o_orderdate', 'o_totalprice', 'l_quantity']]
df_group_2 = df_sort_2 \
    .groupby(['c_custkey', 'o_orderkey', 'c_name', 'o_orderdate', 'o_totalprice']) \
    .agg(
    )
df_group_2 = df_group_2[['sum(l_quantity)']]
df_sort_3 = df_group_2.sort_values(by=['o_totalprice', 'o_orderdate'], ascending=[False, True])
df_sort_3 = df_sort_3[['(sum(l_quantity))']]
df_limit_1 = df_sort_3[['(sum(l_quantity))']]
result = df_limit_1.head(100)

print("---" * 5)
print(time.time() - start_t)
print("---" * 5)

TypeError: Must provide 'func' or tuples of '(column, aggfunc).